Env set up for pandas

In [1]:
import pandas as pd

In [2]:
data1 = pd.read_csv("dataset1.csv")
data2 = pd.read_csv("dataset2.csv")
df = pd.merge(data1, data2).drop(columns=['invoice_id'])

In [3]:
df

,legal_entity,counter_party,rating,status,value,tier
0,L1,C1,1,ARAP,10,1
1,L1,C1,2,ARAP,10,1
2,L1,C1,3,ARAP,20,1
3,L2,C2,2,ARAP,20,2
4,L2,C2,3,ACCR,40,2
5,L3,C3,4,ACCR,30,3
6,L3,C3,3,ACCR,80,3
7,L2,C3,2,ACCR,52,3
8,L3,C3,4,ACCR,35,3
9,L1,C3,6,ARAP,5,3


Creating table with ARAP and ACCR calculated for each individual group

In [4]:
dfGroupedBy =  df.groupby(['legal_entity', 'counter_party', 'tier'])
result = dfGroupedBy.apply(lambda x: pd.Series(dict(
    max_rating=(x.rating).max(),
    sum_ARAP= x.loc[x.status == 'ARAP']['value'].sum(),
    sum_ACCR=x.loc[x.status == 'ACCR']['value'].sum()
)))
result = result.reset_index()
result

,legal_entity,counter_party,tier,max_rating,sum_ARAP,sum_ACCR
0,L1,C1,1,3,40,0
1,L1,C3,3,6,5,0
2,L1,C4,4,6,40,100
3,L2,C2,2,3,20,40
4,L2,C3,3,2,0,52
5,L2,C5,5,6,1000,115
6,L3,C3,3,4,0,145
7,L3,C6,6,6,145,60


Sample data:
legal_entity, counterparty, tier, max(rating by counterparty), sum(value where status=ARAP), sum(value where status=ACCR)
L1,Total, Total, calculated_value, calculated_value,calculated_value
L1, C1, Total,calculated_value, calculated_value,calculated_value
Total,C1,Total,calculated_value, calculated_value,calculated_value
Total,Total,1,calculated_value, calculated_value,calculated_value
L2,Total,Total,calculated_value, calculated_value,calculated_value

grouped by legal_entity, legal_entity + counterparty, counterparty, tier

Create data frame for grouping only with legal_entity

In [8]:
df1 = result.assign(counter_party='total', tier = 'total').groupby(['legal_entity', 'counter_party', 'tier']).agg(max_rating=('max_rating', max), sum_ARAP=('sum_ARAP', sum),sum_ACCR=('sum_ACCR', sum)).reset_index()
df1

,legal_entity,counter_party,tier,max_rating,sum_ARAP,sum_ACCR
0,L1,total,total,6,85,100
1,L2,total,total,6,1020,207
2,L3,total,total,6,145,205


Create data frame for grouping only with legal_entity + counterparty

In [9]:
df2 = result.assign(tier = 'total').groupby(['legal_entity', 'counter_party', 'tier']).agg(max_rating=('max_rating', max), sum_ARAP=('sum_ARAP', sum),sum_ACCR=('sum_ACCR', sum)).reset_index()
df2

,legal_entity,counter_party,tier,max_rating,sum_ARAP,sum_ACCR
0,L1,C1,total,3,40,0
1,L1,C3,total,6,5,0
2,L1,C4,total,6,40,100
3,L2,C2,total,3,20,40
4,L2,C3,total,2,0,52
5,L2,C5,total,6,1000,115
6,L3,C3,total,4,0,145
7,L3,C6,total,6,145,60


Create data frame for grouping only with counterparty

In [10]:
df3 = result.assign(legal_entity = 'total', tier = 'total').groupby(['legal_entity', 'counter_party', 'tier']).agg(max_rating=('max_rating', max), sum_ARAP=('sum_ARAP', sum),sum_ACCR=('sum_ACCR', sum)).reset_index()
df3

,legal_entity,counter_party,tier,max_rating,sum_ARAP,sum_ACCR
0,total,C1,total,3,40,0
1,total,C2,total,3,20,40
2,total,C3,total,6,5,197
3,total,C4,total,6,40,100
4,total,C5,total,6,1000,115
5,total,C6,total,6,145,60


Create data frame for grouping only with tier

In [11]:
df4 = result.assign(legal_entity = 'total', counter_party = 'total').groupby(['legal_entity', 'counter_party', 'tier']).agg(max_rating=('max_rating', max), sum_ARAP=('sum_ARAP', sum),sum_ACCR=('sum_ACCR', sum)).reset_index()
df4

,legal_entity,counter_party,tier,max_rating,sum_ARAP,sum_ACCR
0,total,total,1,3,40,0
1,total,total,2,3,20,40
2,total,total,3,6,5,197
3,total,total,4,6,40,100
4,total,total,5,6,1000,115
5,total,total,6,6,145,60


Concat all data frames to obtain results and save as csv as instructed

In [14]:
pd.concat([df1, df2, df3, df4]).to_csv('pandas_output.csv')